In [1]:
from numpy import *

In [2]:
N_train = 1200
from numpy.random import choice

In [3]:
one_indexes = choice(a=N_train, size=N_train//2, replace=False)

In [4]:
X = zeros((1200,20,1))
Y = zeros((1200,1))

In [5]:
X[one_indexes, 0] = 1
Y[one_indexes] = 1

In [14]:
X_train, X_test = X[:1000,:,:], X[1000:,:,:]
Y_train, Y_test = Y[:1000,:], Y[1000:,:]

In [7]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return array(windows), array(windows_y)

In [22]:
X_train, Y_train = prepare_sequences(X_train, Y_train, 10)
X_test , Y_test  = prepare_sequences(X_test,  Y_test,  10)

In [17]:
X_train.shape

(1000, 20, 1)

In [18]:
from keras.layers import Dense, LSTM 
from keras.models import Sequential

In [28]:
model = Sequential()
model.add(LSTM(10, input_shape=(10,1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=10, nb_epoch=15,
          validation_data=(X_test, Y_test), shuffle=False)
score, acc = model.evaluate(X_test, Y_test, batch_size=10, verbose=1)

ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
- If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
- If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.

In [25]:
score, acc

(0.6601455401290547, 0.5590909103778275)

In [15]:
print('Build STATEFUL model...')
model = Sequential()
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Build STATEFUL model...


In [16]:
print('Train...')
for epoch in range(15):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(len(X_train)):
        y_true = y_train[i]
        for j in range(max_len):
            tr_loss, tr_acc = model.train_on_batch(np.expand_dims(np.expand_dims(X_train[i][j], axis=1), axis=1),
                                                   np.array([y_true]))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()

    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(len(X_test)):
        for j in range(max_len):
            te_loss, te_acc = model.test_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1),
                                                  y_test[i])
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

        for j in range(max_len):
            y_pred = model.predict_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1))
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')

Train...


NameError: name 'y_train' is not defined